In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)

In [2]:
gs.init(backend=gs.cpu, seed=42)

[Genesis] [12:39:43] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [12:39:43] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [12:39:44] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [12:39:47] [INFO] Running on [13th Gen Intel(R) Core(TM) i7-13800H] with backend gs.cpu. Device memory: 31.59 GB.
[Genesis] [12:39:47] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: 42, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.


In [3]:

scene = gs.Scene(
    show_viewer = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
        max_FPS       = 60,
    ),
    sim_options=gs.options.SimOptions(
        dt = 0.01,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
    renderer=gs.renderers.Rasterizer(),
)

plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)
cube_size = (0.1, 0.07, 0.03)
cube_pos = (0.7, 0, 0.015)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
    ),
    surface=gs.surfaces.Default(
        color = (0.8, 0.1 , 0.1),   
    )
)
cube.surface.color = (0.8, 0.1 , 0.1)

cylinder_pos = (0.7, -0.2, 0.03)
cylinder_quat = (0, -np.sqrt(2)/2, 0, np.sqrt(2)/2)
cylinder_radius = 0.03
cylinder_height = 0.1

cylinder = scene.add_entity(
    gs.morphs.Cylinder(
        radius = cylinder_radius,
        height = cylinder_height,
        pos    = cylinder_pos,
        quat   = cylinder_quat,
    ),
    surface=gs.surfaces.Default(
        color = (0.1, 0.8 , 0.1),   
    )
)
cylinder.surface.color = (0.2, 0.8, 0.2)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)


[Genesis] [12:40:04] [INFO] Scene <fb1326c> created.
[Genesis] [12:40:04] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <9eba5f4>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [12:40:04] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <07427e1>, morph: <gs.morphs.MJCF(file='d:\envs\pygen\Lib\site-packages\genesis\assets\xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [12:40:07] [WARNING] (MJCF) Tendon not supported
[Genesis] [12:40:07] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [12:40:07] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.
[Genesis] [12:40:07] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [12:40:07] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias.

In [4]:

scene.build()
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
# Note: the following values are tuned for achieving best behavior with Franka
# Typically, each new robot would have a different set of parameters.
# Sometimes high-quality URDF or XML file would also provide this and will be parsed.
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
)

# render rgb, depth, segmentation, and normal
# rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)


[Genesis] [12:40:13] [INFO] Building scene <fb1326c>...
[Genesis] [12:40:17] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [12:40:17] [WARNING] Constraint solver time constant was increased to avoid numerical instability (from `0.005` to `0.02`). Decrease simulation timestep to avoid altering the original value.
[Genesis] [12:40:36] [INFO] Compiling simulation kernels...
[Genesis] [12:45:28] [INFO] Building visualizer...
[Genesis] [12:45:35] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [154]:
scene.reset()

[Genesis] [14:38:48] [INFO] Resetting Scene <fb1326c>.


In [155]:
# franka is in ready position
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])
franka.ready_qpos = qr
franka.set_qpos(franka.ready_qpos)
scene.step()

[Genesis] [14:38:50] [INFO] Running at 0.57 FPS.


In [6]:
# get cubes prperties
cube_pos = cube.get_pos()
print(f"cube_pos: {cube_pos}")
cube_rot = cube.get_quat()
print(f"cube_rot: {cube_rot}")
cube_mass = cube.get_mass()
print(f"cube_mass: {cube_mass}")
cube_vertices = cube.get_verts()
print(f"cube_vertices: {cube_vertices}")

cube_pos: tensor([0.7000, 0.0000, 0.0140])
cube_rot: tensor([1., 0., 0., 0.])
cube_mass: 0.04200000000000001
cube_vertices: tensor([[ 0.6500, -0.0350, -0.0010],
        [ 0.6500, -0.0350,  0.0290],
        [ 0.6500,  0.0350, -0.0010],
        [ 0.6500,  0.0350,  0.0290],
        [ 0.7500, -0.0350, -0.0010],
        [ 0.7500, -0.0350,  0.0290],
        [ 0.7500,  0.0350, -0.0010],
        [ 0.7500,  0.0350,  0.0290]])


In [7]:
# get cylinder properties
cylinder_pos = cylinder.get_pos()
print(f"cylinder_pos: {cylinder_pos}")
cylinder_rot = cylinder.get_quat()
print(f"cylinder_rot: {cylinder_rot}")
cylinder_mass = cylinder.get_mass()
print(f"cylinder_mass: {cylinder_mass}")
# cylinder_vertices = cylinder.get_verts()
# print(f"cylinder_vertices: {cylinder_vertices}")

cylinder_pos: tensor([ 0.7003, -0.2000,  0.0294])
cylinder_rot: tensor([-1.9586e-20, -7.0313e-01,  1.3379e-20,  7.1106e-01])
cylinder_mass: 0.05618601274064494


In [46]:
def get_obj_screw(object):
    """
    Get the screw parameters of the object
    """
    # get the vertices of the object
    vertices = object.get_verts()
    vertices = vertices.numpy()

    # get the max and min x, y, z coordinates
    max_x = np.max(vertices[:, 0])
    min_x = np.min(vertices[:, 0])
    min_y = np.min(vertices[:, 1])
    min_z = np.min(vertices[:, 2])
    max_y = np.max(vertices[:, 1])
    
    v1 = np.array([max_x, min_y, min_z])  # max x, min y, min z
    v2 = np.array([max_x, max_y, min_z])  # max x, max y, min z
    v3 = np.array([min_x, max_y, min_z])  # min x, max y, min z
    
    if np.linalg.norm(v1-v2) <= np.linalg.norm(v2-v3):
        s = (v1-v2) / np.linalg.norm(v1-v2) # unit vector in the direction of the screw axis
        q = v2 # point on the screw axis
        h = 0 # pitch of the screw axis
    else:
        s = (v2-v3) / np.linalg.norm(v2-v3) # unit vector in the direction of the screw axis
        q = v2 # point on the screw axis
        h = 0 # pitch of the screw axis
    
    pt.check_screw_parameters(q, s, h) # check if the screw parameters are valid
    return s, q, h


In [47]:
def obj_dq(object):
    """
    Get the screw parameters of the object and apply sclerp to the object in dual quaternion
    """
    # get the screw parameters of the object
    s, q, h = get_obj_screw(object)
    
    pq = object.get_qpos().cpu().numpy()

    obj_initial_dq = pt.dual_quaternion_from_pq(pq) # dual quaternion from position and quaternion
    
    Screw_dq = pt.dual_quaternion_from_screw_parameters(q=q, s_axis=s, h=0, theta=-np.pi/2) # dual quaternion from screw parameters that make the object upright
    # intermediate_T = pt.transform_log_from_screw_matrix
    # Screw_T = pt.transform_from_dual_quaternion(Screw_dq) # transform from dual quaternion to transformation matrix
    # print("screw_dq: \n", Screw_dq)
    # print("screw_T: \n", Screw_T)
    obj_goal_dq = pt.concatenate_dual_quaternions(Screw_dq, obj_initial_dq)
    #obj_goal_T = pt.transform_from_dual_quaternion(obj_goal_dq) # transform from dual quaternion to transformation matrix
    # print("obj_goal_dq: \n", obj_goal_dq)
    # print("obj_goal_T: \n", obj_goal_T)
    return obj_initial_dq, obj_goal_dq

In [48]:
def obj_sclerp(object, obj_initial_dq, obj_goal_dq, method, steps):
    """
    Apply sclerp to the object in dual quaternion
    """
    s = np.linspace(0, 1, steps)
    if method == 'linear':
        tau = s
    if method == 'cubic':
        tau = 3*s**2 - 2*s**3
    if method == 'quintic':
        tau = 10*s**3 - 15*s**4 + 6*s**5
        
    # get the dual quaternion of the object
    dq_traj = np.vstack(
            [ptr.dual_quaternions_sclerp(obj_initial_dq, obj_goal_dq, t) for t in tau]
        )  # spherical linear interpolation between two dual quaternions
    # get the position and quaternion from the dual quaternion
    obj_qpos = ptr.pqs_from_dual_quaternions(dq_traj)
    
    return dq_traj, obj_qpos

In [11]:
cube.size = cube_size
cube.size

(0.1, 0.07, 0.03)

In [49]:
def obj_grasp_offset(object, object_initial_dq):
    
    if object == cube:
        length = cube.size[0]
        T_initial = pt.transform_from_dual_quaternion(object_initial_dq) # transform from dual quaternion to transformation matrix
        T_offset = np.array([[1, 0, 0, -length/2 + 0.02],
                             [0, 1, 0, 0],
                             [0, 0, 1, 0],
                             [0, 0, 0, 1]])
        T_ee = np.array([[1, 0, 0, 0],
                         [0, -1, 0, 0],
                         [0, 0, -1, 0.11],
                         [0, 0, 0, 1]])
        T_grasp = T_initial @ T_offset @ T_ee # transformation matrix from the object to the end effector
        
        return T_grasp

In [50]:
def obj_grasp(object):
    """
    grasping the object
    """
    # get the screw parameters of the object
    s, q, h = get_obj_screw(object)
    
    # get the dual quaternion of the object
    obj_initial_dq, obj_goal_dq = obj_dq(object)

    T_grasp = obj_grasp_offset(object, obj_initial_dq) 
    pq_grasp = pt.pq_from_transform(T_grasp) # position and quaternion from transformation matrix
    return pq_grasp

In [102]:
# get end effectors position and quaternion
franka_joint = franka.get_qpos()
franka_joint

tensor([ 4.1442e-04,  8.2520e-01, -6.3591e-04, -1.6923e+00,  2.3042e-04,
         2.5006e+00,  7.8517e-01,  3.4882e-02,  3.4882e-02])

In [113]:
obj_grasp(cube) # get the grasping position and quaternion of the cube

array([ 6.70004009e-01, -1.20054944e-04,  1.24893889e-01,  1.25697726e-04,
        9.99999992e-01,  1.18859324e-05, -8.25255932e-06])

In [139]:
franka_p, franka_q = franka.forward_kinematics(franka_joint, links_idx_local=[8])
pq = np.concatenate([franka_p, franka_q], axis=1).reshape(7) # position and quaternion of the end effector

In [140]:
pq.shape

(7,)

In [141]:
pt.dual_quaternion_from_pq(pq) # dual quaternion from position and quaternion

array([ 1.64472513e-04,  9.99964595e-01,  1.06394300e-05, -8.42391141e-03,
       -3.32334518e-01,  5.46132405e-05,  6.02510299e-02,  7.03296078e-05])

In [142]:
def franka_sclerp(object, robot, method, steps):
    franka_initial_p, franka_initial_q = robot.forward_kinematics(robot.get_qpos(), links_idx_local=[8])
    franka_initial_pq = np.concatenate((franka_initial_p, franka_initial_q), axis=1).reshape(7) # position and quaternion of the end effector
    franka_initial_dq = pt.dual_quaternion_from_pq(franka_initial_pq) # dual quaternion from position and quaternion
    s,q,h = get_obj_screw(object)
    Screw_dq = pt.dual_quaternion_from_screw_parameters(q=q, s_axis=s, h=0, theta=-np.pi/2) # dual quaternion from screw parameters that make the object upright
    franka_goal_dq = pt.concatenate_dual_quaternions(Screw_dq, franka_initial_dq)
    
    s = np.linspace(0, 1, steps)
    if method == 'linear':
        tau = s
    if method == 'cubic':
        tau = 3*s**2 - 2*s**3
    if method == 'quintic':
        tau = 10*s**3 - 15*s**4 + 6*s**5
        
    # get the dual quaternion of the object
    franka_dq_traj = np.vstack(
            [ptr.dual_quaternions_sclerp(franka_initial_dq, franka_goal_dq, t) for t in tau]
        )  # spherical linear interpolation between two dual quaternions
    # get the position and quaternion from the dual quaternion
    franka_pqs = ptr.pqs_from_dual_quaternions(franka_dq_traj)
    return franka_pqs
    

In [160]:
franka_pqs = franka_sclerp(cube, franka, 'linear', 100) # get the trajectory of the end effector in dual quaternion
franka_pqs.shape

(100, 7)

In [156]:
pq_grasp_cube = obj_grasp(cube)
pq_grasp_cube[0:3]


array([0.66999999, 0.        , 0.124019  ])

In [157]:
# move to pre-grasp pose
end_effector = franka.get_link('hand')
q_pregrasp = franka.inverse_kinematics(
    link = end_effector,
    pos = pq_grasp_cube[:3],
    quat = pq_grasp_cube[3:7], # quaternion from the transformation matrix
)

In [158]:
franka.control_dofs_position(q_pregrasp)
for _ in range(300):
    scene.step()

[Genesis] [14:39:39] [INFO] Running at 0.24 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.25 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.27 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.28 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.30 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.31 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.33 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.35 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.36 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.38 FPS.
[Genesis] [14:39:39] [INFO] Running at 0.40 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.42 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.44 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.47 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.49 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.52 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.54 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.57 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.60 FPS.
[Genesis] [14:39:40] [INFO] Running at 0.63 FPS.
[Genesis] [14:39:40]

In [159]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)

for _ in range(100):
    scene.step()

[Genesis] [14:39:53] [INFO] Running at 9.13 FPS.


[Genesis] [14:39:53] [INFO] Running at 9.53 FPS.
[Genesis] [14:39:53] [INFO] Running at 9.48 FPS.
[Genesis] [14:39:53] [INFO] Running at 9.50 FPS.
[Genesis] [14:39:53] [INFO] Running at 9.60 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.70 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.74 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.08 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.14 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.21 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.35 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.38 FPS.
[Genesis] [14:39:54] [INFO] Running at 9.43 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.31 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.39 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.49 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.40 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.33 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.38 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.32 FPS.
[Genesis] [14:39:55] [INFO] Running at 9.35 FPS.
[Genesis] [14:39:55]

In [92]:
scene.reset()
franka.set_qpos(franka.ready_qpos)
scene.step()

[Genesis] [13:59:15] [INFO] Resetting Scene <fb1326c>.
[Genesis] [13:59:15] [INFO] Running at 0.02 FPS.


In [161]:
init_q = franka.get_qpos()
qs = []
for pq in franka_pqs:
    q = franka.inverse_kinematics(
        link=end_effector,
        pos=pq[:3],
        quat=pq[3:7],
        init_qpos=init_q,
    )
    init_q = q
    qs.append(q)

In [162]:
qs = np.array(qs)
for q in qs:
    franka.set_qpos(q[:-2], motors_dof)
    scene.step()

for _ in range(100):
    scene.step()    

[Genesis] [14:41:05] [INFO] Running at 0.32 FPS.
[Genesis] [14:41:05] [INFO] Running at 0.34 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.35 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.37 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.39 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.41 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.43 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.45 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.47 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.50 FPS.
[Genesis] [14:41:06] [INFO] Running at 0.52 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.55 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.58 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.61 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.64 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.67 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.70 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.74 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.78 FPS.
[Genesis] [14:41:07] [INFO] Running at 0.82 FPS.
[Genesis] [14:41:07]

In [16]:
# calling functions for cube_sclerp
scene.reset()
s, q, h = get_obj_screw(cube)
cube_initial_dq, Cube_goal_dq = obj_dq(cube)
dq_traj, obj_qpos = obj_sclerp(cube, cube_initial_dq, Cube_goal_dq, 'linear', 100)
dq_traj.shape

[Genesis] [17:49:27] [INFO] Resetting Scene <2580b40>.


(100, 8)

In [ ]:
# cube sclerp motion
scene.reset()
for i in range (obj_qpos.shape[0]):
    franka.set_qpos(qr)
    cube.set_qpos(obj_qpos[i])
    scene.step()


[Genesis] [17:49:36] [INFO] Resetting Scene <2580b40>.
[Genesis] [17:49:36] [INFO] Running at 0.05 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.06 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.06 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.06 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.07 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.07 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.07 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.08 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.08 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.09 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.09 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.09 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.10 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.10 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.11 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.12 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.12 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.13 FPS.
[Genesis] [17:49:36] [INFO] Running at 0.14 FPS.
[Genesis] [17:

In [22]:
# calling functions for cylinder_sclerp
scene.reset()
s, q, h = get_obj_screw(cylinder)
cyl_initial_dq, cyl_goal_dq = obj_dq(cylinder)
dq_traj, obj_qpos = obj_sclerp(cylinder, cyl_initial_dq, cyl_goal_dq, 'linear', 100)
dq_traj.shape

[Genesis] [17:52:28] [INFO] Resetting Scene <2580b40>.


(100, 8)

In [ ]:
# cylinder sclerp motion
scene.reset()
for i in range (obj_qpos.shape[0]):
    franka.set_qpos(qr)
    cylinder.set_qpos(obj_qpos[i])
    scene.step()

[Genesis] [17:51:21] [INFO] Resetting Scene <2580b40>.
[Genesis] [17:51:22] [INFO] Running at 0.87 FPS.
[Genesis] [17:51:22] [INFO] Running at 0.92 FPS.
[Genesis] [17:51:22] [INFO] Running at 0.97 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.02 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.07 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.12 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.18 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.24 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.30 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.36 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.43 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.50 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.58 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.66 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.74 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.83 FPS.
[Genesis] [17:51:22] [INFO] Running at 1.92 FPS.
[Genesis] [17:51:22] [INFO] Running at 2.02 FPS.
[Genesis] [17:51:22] [INFO] Running at 2.12 FPS.
[Genesis] [17:

In [ ]:
end_effector = franka.get_link('hand')
left_finger = franka.get_link('left_finger')
right_finger = franka.get_link('right_finger')

left_finger.get_pos(), right_finger.get_pos(), end_effector.get_pos()

In [ ]:

# get the end-effector link
end_effector = franka.get_link('hand')
above_box = cube.get_pos().cpu().numpy() + np.array([-cube_size[0]/2 + 0.02, 0, 0.11 + 0.1])
# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = above_box,
    quat = np.array([0, 1, 0, 0]),
)
# # gripper open pos
# qpos[-2:] = 0.04
# path = franka.plan_path(
#     qpos_goal     = qpos,
#     num_waypoints = 200, # 2s duration
# )

In [ ]:
franka.get_qpos(), qpos

# find straight path in joint space
s = np.linspace(0, 1, 200)
current_pose = franka.get_qpos().cpu().numpy()
goal_pos = qpos.cpu().numpy()
path = current_pose + s[:, None] * (goal_pos - current_pose)


In [ ]:
for qpos in path:
    # set qpos
    franka.control_dofs_position(qpos)
    # step simulation
    scene.step()

In [ ]:
qpos

In [ ]:
# # execute the planned path
# for waypoint in path:
#     franka.control_dofs_position(waypoint)
#     scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


In [ ]:
side_box = above_box + np.array([0.0, 0.0, -0.1])


# reach
qpos_reach = franka.inverse_kinematics(
    link = end_effector,
    pos  = side_box,
    quat = np.array([0, 1, 0, 0]),
)
# path = franka.plan_path(
#     qpos_goal     = qpos_reach,
#     num_waypoints = 100, # 1s duration
# )

# for waypoint in path:
franka.control_dofs_position(qpos_reach)
    # scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:



# grasp
franka.control_dofs_position(qpos_reach[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in range(100):
    scene.step()


In [ ]:

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(200):
    scene.step()

In [ ]:
# open
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:
franka.get_dofs_force()

In [ ]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

cube_pos, cube_quat

In [ ]:
cube_pos = cube.get_pos().cpu().numpy()
cube_quat = cube.get_quat().cpu().numpy()

qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=cube_pos + np.array([0, 0, 0.3]),
    quat=np.array([0, 1, 0, 0]),
)

franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
for i in range(100):
    scene.step()
# allow robot to reach the last waypoint
for i in range(100):
    scene.step()

In [ ]:

cam.start_recording()
import numpy as np

for i in range(120):
    scene.step()
    cam.set_pose(
        pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
        lookat = (0, 0, 0.5),
    )
    cam.render()
cam.stop_recording(save_to_filename='video.mp4', fps=60)